In [43]:
import numpy as np
import pandas as pd
import sklearn

In [44]:
df=pd.read_csv("diabetes.csv")

In [45]:
df.shape

(768, 9)

In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [46]:
x = df.iloc[:,0:-1].values
y = df.iloc[:,-1].values

In [47]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [48]:
x = scaler.fit_transform(x)

In [49]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [96]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [51]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim = 8))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )

In [52]:
model.fit(X_train,y_train,batch_size = 32,epochs = 10, validation_data = (X_test,y_test))

Epoch 1/10
20/20 [==============================] - 1s 12ms/step - loss: 0.7207 - accuracy: 0.5423 - val_loss: 0.6725 - val_accuracy: 0.6104
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6554 - accuracy: 0.6759 - val_loss: 0.6192 - val_accuracy: 0.6948
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6090 - accuracy: 0.7215 - val_loss: 0.5859 - val_accuracy: 0.7208
Epoch 4/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5738 - accuracy: 0.7313 - val_loss: 0.5616 - val_accuracy: 0.7532
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5475 - accuracy: 0.7427 - val_loss: 0.5419 - val_accuracy: 0.7662
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5258 - accuracy: 0.7410 - val_loss: 0.5290 - val_accuracy: 0.7727
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5102 - accuracy: 0.7508 - val_loss: 0.5211 - val_accuracy: 0.7532
Epoch 8/10
20/20 [=

In [19]:
#1.how to select appropriate optimiser
#2.number of nodes in a layer
#3.how to select number of hidden layers
#4.All in one model

In [22]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.4 MB/s eta 0:00:00


In [53]:
import kerastuner as kt

In [62]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32,activation = 'relu', input_dim = 8))
  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['sgd','adadelta','rmsprop','adam']), loss = 'binary_crossentropy', metrics =['accuracy'])

  return model

In [63]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 5)

In [64]:
tuner.search(X_train,y_train,epochs = 5, validation_data = (X_test,y_test))

In [65]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [66]:
model  = tuner.get_best_models(num_models=1)[0]

In [67]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.fit(X_train,y_train,batch_size=32,epochs = 100, initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5395 - accuracy: 0.7541 - val_loss: 0.5661 - val_accuracy: 0.7143
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5214 - accuracy: 0.7590 - val_loss: 0.5495 - val_accuracy: 0.7403
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5067 - accuracy: 0.7606 - val_loss: 0.5377 - val_accuracy: 0.7403
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4952 - accuracy: 0.7671 - val_loss: 0.5278 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4851 - accuracy: 0.7671 - val_loss: 0.5191 - val_accuracy: 0.7597
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4772 - accuracy: 0.7720 - val_loss: 0.5133 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7720 - val_loss: 0.5095 - val_accuracy: 0.7727
Epoch 14

In [73]:
#what should be the number of neurons in a particular layer
def build_model(hp):

  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step = 8)
  model.add(Dense(units=units,activation = 'relu',input_dim = 8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer = 'adam', loss='binary_crossentropy',metrics = ['accuracy'])

  return model

In [76]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials = 5,
                        directory = 'mydir')

In [77]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))


Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.6883116960525513

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 11s


In [78]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [79]:
model = tuner.get_best_models(num_models=1)[0]

In [80]:
model.fit(X_train,y_train,batch_size=32,epochs = 100, initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.4863 - accuracy: 0.7655
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4724 - accuracy: 0.7687
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4631 - accuracy: 0.7704
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4559 - accuracy: 0.7818
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4508 - accuracy: 0.7866
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4474 - accuracy: 0.7866
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4431 - accuracy: 0.7883
Epoch 14/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4407 - accuracy: 0.7964
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4377 - accuracy: 0.7932
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4354 - accuracy: 0.7883
Epo

In [81]:
#How to select number of layers?

In [84]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(104,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10,step = 1)):
    model.add(Dense(104,activation='relu'))

  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])

  return model

In [85]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials= 5,
                        directory = 'mydir',
                        project_name='num_layers')

In [86]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7272727489471436

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 17s


In [87]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [88]:
model = tuner.get_best_models(num_models=1)[0]

In [89]:
model.fit(X_train,y_train,epochs = 100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 14ms/step - loss: 0.4573 - accuracy: 0.7899 - val_loss: 0.5058 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4386 - accuracy: 0.7932 - val_loss: 0.5155 - val_accuracy: 0.7597
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4281 - accuracy: 0.8046 - val_loss: 0.5142 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4190 - accuracy: 0.8078 - val_loss: 0.5277 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4152 - accuracy: 0.8029 - val_loss: 0.5405 - val_accuracy: 0.7468
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4029 - accuracy: 0.8029 - val_loss: 0.5297 - val_accuracy: 0.7403
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3958 - accuracy: 0.8160 - val_loss: 0.5402 - val_accuracy: 0.7532
Epoch 14

In [107]:
#hypertuning multiple parameters

def build_model(hp):
  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value = 1,max_value = 10,step = 1)):
    if counter == 0:
      model.add(Dense(hp.Int('units'+str(i),min_value = 8, max_value = 128,step = 8),
                      activation = hp.Choice("activation"+str(i),values=['relu','tanh','sigmoid','elu']),
                      input_dim = 8))
      model.add(Dropout(hp.Choice('Dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int('units'+str(i),min_value = 8, max_value = 128,step = 8),
                      activation = hp.Choice("activation"+str(i),values=['relu','tanh','sigmoid','elu'])))
      model.add(Dropout(hp.Choice('Dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1

  model.add(Dense(1,activation = 'sigmoid'))

  model.compile(optimizer = hp.Choice('optimizer',values=['rmsprop','adam','sgd','adadelta','nadam']),
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

  return model

In [108]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydir',
                        project_name = 'final_1')

In [109]:
tuner.search(X_train,y_train,epochs = 5,validation_data = (X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7272727489471436
Total elapsed time: 00h 00m 08s


In [110]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'units0': 112,
 'activation0': 'relu',
 'Dropout0': 0.6,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'relu',
 'Dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'Dropout2': 0.1}

In [111]:
model = tuner.get_best_models(num_models=1)[0]

In [112]:
model.fit(X_train,y_train,epochs = 200,initial_epoch = 6, validation_data = (X_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 14ms/step - loss: 0.5306 - accuracy: 0.7215 - val_loss: 0.5391 - val_accuracy: 0.7338
Epoch 8/200
20/20 [==============================] - 0s 7ms/step - loss: 0.5185 - accuracy: 0.7443 - val_loss: 0.5320 - val_accuracy: 0.7532
Epoch 9/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5194 - accuracy: 0.7264 - val_loss: 0.5270 - val_accuracy: 0.7597
Epoch 10/200
20/20 [==============================] - 0s 8ms/step - loss: 0.4886 - accuracy: 0.7557 - val_loss: 0.5214 - val_accuracy: 0.7532
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4774 - accuracy: 0.7720 - val_loss: 0.5185 - val_accuracy: 0.7403
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4900 - accuracy: 0.7459 - val_loss: 0.5162 - val_accuracy: 0.7532
Epoch 13/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4879 - accuracy: 0.7622 - val_loss: 0.5175 - val_accuracy: 0.7403
Epoch 14